# Player Analysis for BPL 10

Author: Cearlmau

This file breaks down the player distribution of BPL teams based on team and level
at various points in the BPL 10 event. The goal of this data analysis is to look at
whether or not the team distributions had an effect on overall team player levels, and
how that may contribute to a team's success.

Special thanks to Moowiz for providing the data.

In [35]:
#Libraries and Data Sources
import csv
import pandas as pd
import plotly
import datetime

level_data = pd.read_csv("../../data/bpl-10_level_data.csv")
teams_data = pd.read_csv("../../data/bpl-10_teams.csv")

## Background:

Path of Exile is a popular game whose premise revolves around the player killing enemies using a preset character with an extensive amount of customization. The killed enemies generate loot and other bonuses which the player can use to augment and upgrade their character. Path of Exile is an online game, which means that players can interact with one another, trading each items and/or playing in the same instance as one another.

BPL is an quarterly event that coincides with each major update that Path of Exile receives. Around 2000 players is placed inside an isolated and newly created instance (referred to as a "league") where they must work together with their designated team to acquire items and accomplish objectives("bounties") defined by the organizers. Doing so accumulates points towards their team total, and at the end the teams are ranked based on how many points they have acquired over the four-day event. There are 4 teams for BPL 10.

Each preset character has a specific upgrade called an "ascendancy". This allows the character to gain unique powers that are inaccesible to other presets. Each character is permitted only one ascendancy. To balance out the teams, the organizers have made it so that each team can only have a subset of the presets. One of the ascendancies, "Sabateur", is banned.

One thing to note is that ascendancy bonuses are acquired in portions. To be fully ascended is to grab all possible upgrades related to that ascendancy.

Each character also has a stage-based development referred to as "levels". Characters will start off weak. As they kill enemies they acquire numerical reward ("experience points"). When they have acquired sufficient experience points, they will "level up", and achieve the next stage of their character progression. In Path of Exile, a character can have levels from 1 to 100, inclusive of both ends. Typically, having more levels mean that your character becomes stronger.

The playthrough of a character is broken down into two parts: campaign, where the character goes through a story-driven progression, and endgame, where the player can tackle various content unavailable in campaign. All of BPL's bounties occur in the endgame. 

A large portion of the endgame takes place in the Atlas, where players can enter special areas to fight harder enemies and acquire more rewarding loot. Completing sections of the Atlas awards players with the power to customize the Atlas in the form of "atlas passives". Note that is it not required that a player needs all atlas passives to participate in the endgame, but having more gives the player more options in their playstyle.

The data is broken down into two csvs. 
**bpl-10_teams.csv** contains information at the end of the event. It contains finalized data of all the players, including level, whether or not a player has fully ascended, and the number of atlas passives they have acquired. 
**bpl-10_level_data.csv** contains progressive tracking of every single character's levelling over the course of the event. Snapshots of each character are taken at approximately every 15 minutes. Each entry contains the player, their team name, their character level, and time of snapshot.

For anonymity and ethics, player names are replaced with hashed values.


In [5]:
'''
The first thing we have to do is clean up the data.
While there are no duplicates or missing values,
the levelling data is inconsistent. Since data collection
happened in real time, the timestamps can differ by seconds,
even within the same snapshot.
'''
level_data.astype({'team': 'string', 'char_name_hash': 'string'}).dtypes
level_data["time_fetched"]



team              string
char_level         int64
char_name_hash    string
time_fetched      object
dtype: object

In [37]:
test_row = level_data["time_fetched"].sample()
print(test_row.values)
format = "%Y-%m-%d %H:%M:%S"
tm = datetime.datetime.strptime(test_row.values[0], format)
discard = datetime.timedelta(minutes=tm.minute % 15,
                             seconds=tm.second
                             )
tm -= discard
if discard >= datetime.timedelta(minutes=5):
    tm += datetime.timedelta(minutes=15)
print(tm)

['2023-01-17 7:59:03']
2023-01-17 08:00:00
